In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
  
from tensorflow.keras import models, layers, utils 
from sklearn import metrics, preprocessing
import os
import gc
import sys

import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate



from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from tensorflow.keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack




sns.set_style("darkgrid")

gc.collect()


c:\Users\Shehab\.conda\envs\tensor001\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


22

## recommender system 
the two main functions are 
get_similar_movies(movie_title, min_ratings_count = 0)

recommend_movies(user_id , n_movies= 10)


In [14]:
path = "D:\\netflix dataset\\netflix-prize-data"


In [15]:
start = datetime.now()
if not os.path.isfile('data.csv'):
    data = open('data.csv', mode='w')
    row = list()
    files = os.listdir(path)
    print(files)
    for file in files:
        if file.startswith('combined_data'):
        # if file.startswith('combined_data_1'):   # use this line to process only one file
            print("Reading ratings from {}\n".format(file))
            with open(os.path.join(path,file)) as f:
                for line in f:
                    line = line.strip()
                    if line.endswith(':'):
                        # All below are ratings for this movie, until another movie appears.
                        movie_id = line.replace(':', '')
                    else:
                        row = [x for x in line.split(',')]
                        row.insert(0, movie_id)
                        data.write(','.join(row))
                        data.write('\n')
    data.close()
print('Time taken :', datetime.now() - start)


Time taken : 0:00:00


In [43]:
df = pd.read_csv('data.csv', names=['Movie_Id', 'Cust_Id', 'Rating', 'Date'])


In [44]:
df.head()


,Movie_Id,Cust_Id,Rating,Date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  int64 
 1   Cust_Id   int64 
 2   Rating    int64 
 3   Date      object
dtypes: int64(3), object(1)
memory usage: 3.0+ GB


In [46]:
df['Movie_Id'] = df['Movie_Id'].astype(np.uint32)
df['Cust_Id'] = df['Cust_Id'].astype(np.uint32)
df['Rating'] = df['Rating'].astype(np.uint8)
df.drop(['Date'], axis=1, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 862.4 MB


In [47]:
# f = ['count', 'mean']

# df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
# df_movie_summary.index = df_movie_summary.index.map(int)
# movie_benchmark = round(df_movie_summary['count'].quantile(0.7), 0)
# drop_movie_list = df_movie_summary[df_movie_summary['count']
#                                    < movie_benchmark].index
# print(len(drop_movie_list))
# print('Movie minimum times of review: {}'.format(movie_benchmark))

# ### i decided not to drop any customer data since later when we will recommend movies to a customer, we will need to know the customer's rating history

# df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
# df_cust_summary.index = df_cust_summary.index.map(int)
# cust_benchmark = round(df_cust_summary['count'].quantile(0.7), 0)
# drop_cust_list = df_cust_summary[df_cust_summary['count']
#                                  < cust_benchmark].index

# print('Customer minimum times of review: {}'.format(cust_benchmark))

# print('Original Shape: {}'.format(df.shape))
# df = df[~df['Movie_Id'].isin(drop_movie_list)]
# # df.drop(df['Movie_Id'].isin(drop_movie_list), axis=0, inplace=True)
# df = df[~df['Cust_Id'].isin(drop_cust_list)]
# print('After Trim Shape: {}'.format(df.shape))


# print("number of movies deleted is " , len(drop_movie_list))
# # print("number of customers deleted is " , len(drop_cust_list))


# Filter sparse movies
min_movie_ratings = 10000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings
print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))


Shape User-Ratings unfiltered:	(100480507, 3)
Shape User-Ratings filtered:	(60546559, 3)


In [48]:
# print(len(drop_movie_list))


In [49]:
# del df_cust_summary, df_movie_summary, drop_movie_list, drop_cust_list
# gc.collect()
print(df.info())
df.index = range(len(df))

print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 862.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 862.4 MB
None


In [50]:
df_title = pd.read_csv("D:\\netflix dataset\\netflix-prize-data\\movie_titles.csv", encoding="ISO-8859-1",
                        header=None, names=['Movie_Id', 'Year', 'Name'], usecols=[0, 1, 2])
df_title.set_index('Movie_Id', inplace=True)
df_title['Year'].fillna(0, inplace=True)
df_title['Year'] = df_title['Year'].astype(int)
df_title.head(10)


,Year,Name
Movie_Id,,
1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW
6,1997,Sick
7,1992,8 Man
8,2004,What the #$*! Do We Know!?
9,1991,Class of Nuke 'Em High 2


In [81]:
df_pivot = pd.pivot_table(df_filterd, values='Rating',
                      index='Cust_Id', columns='Movie_Id')

print(df_pivot.shape)


(150245, 2042)


In [82]:
df_pivot.info()
# nearly 6 gb of ram is used by pivot table

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 150245 entries, 6 to 2649429
Columns: 2042 entries, 8 to 17764
dtypes: float64(2042)
memory usage: 2.3 GB


In [83]:
# min count is min count of ratings given by a customer
def get_similar_movies(movie_title, min_ratings_count=0):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df_title.index[df_title['Name'] == movie_title][0])
    target = df_pivot[i]
    similar_to_target = df_pivot.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Name', 'count', 'mean']]
    corr_target.rename(columns={"Name": "movie name", "count": "number of ratings"  , "mean": "average rating" ,
                                "PearsonR": "Pearson correlation coefficient"}, inplace=True)
    
    print(corr_target[corr_target['number of ratings']>min_ratings_count][1:11].to_string(index=False))

In [84]:
get_similar_movies("Lord of the Rings: The Fellowship of the Ring", 30000)
# you can see it outputs similar movies 


For movie (Lord of the Rings: The Fellowship of the Ring)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient                                                          movie name  number of ratings  average rating
                        0.809397                                   Lord of the Rings: The Two Towers             151245        4.460676
                        0.756525                           Lord of the Rings: The Return of the King             134284        4.545121
                        0.691291 The Lord of the Rings: The Fellowship of the Ring: Extended Edition              73422        4.716611
                        0.658069                 Lord of the Rings: The Two Towers: Extended Edition              74912        4.702611
                        0.630302         Lord of the Rings: The Return of the King: Extended Edition              73335        4.723270
                        0.259625                               

In [85]:
get_similar_movies("Fight Club")
get_similar_movies("Fight Club", 80000)
# here it actually ouputs movies of the same genre or the the same director
# and min count produces better recommendations if used correctly

For movie (Fight Club)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient                            movie name  number of ratings  average rating
                        0.380176                            12 Monkeys              76475        3.776084
                        0.378462                                Snatch              68818        3.848005
                        0.364801                               Memento             127318        3.851427
                        0.357735                          Pulp Fiction             152078        4.094235
                        0.348350                         Trainspotting              24446        3.885953
                        0.343353 Eternal Sunshine of the Spotless Mind             110159        3.695304
                        0.333795                        Reservoir Dogs              90450        4.003847
                        0.326206                              Sin 

In [86]:
get_similar_movies("Saving Private Ryan", 0)
# here it ouputs similar war movies


For movie (Saving Private Ryan)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient       movie name  number of ratings  average rating
                        0.348560  Black Hawk Down              77597        3.859608
                        0.344548       Braveheart             135601        4.294423
                        0.328656        Apollo 13              98116        4.082576
                        0.321469        Gladiator             150592        4.202713
                        0.320689     Forrest Gump             181508        4.299910
                        0.315668 Band of Brothers              37514        4.513328
                        0.315341          Platoon              64175        3.986038
                        0.304082 We Were Soldiers              64917        4.005299
                        0.295455 Schindler's List             101141        4.458004
                        0.285864        Cast Away      

In [89]:
# del df_pivot
# del df
gc.collect()

15

### collaborative filtering
Singular Value Decomposition (SVD)

In [91]:
# first we can observe like movies of a customer
def get_prev_liked_movies(user_id, num_movies=10 , verbose=False):
    # liked_movies = df[(df['Cust_Id'] == user_id) & (df['Rating'] == rating)]
    liked_movies = df_filterd[(df_filterd['Cust_Id'] == user_id)]
    liked_movies.set_index('Movie_Id', inplace=True)
    liked_movies = liked_movies.join(df_title)[['Name', 'Year', 'Rating']]
    # print(liked_movies.unique())
    if verbose:
        print("Liked movies by user {} are : ".format(user_id))
        print(liked_movies.head(num_movies).to_string(index=False))
    # return the id of liked movies
    return liked_movies.index

get_prev_liked_movies(1987434 , verbose=True)


Liked movies by user 1987434 are : 
                      Name  Year  Rating
What the #$*! Do We Know!?  2004       4
           Lilo and Stitch  2002       4
    Something's Gotta Give  2003       2
               Dragonheart  1996       3
                   Spartan  2004       2
            Reservoir Dogs  1992       4
          Death to Smoochy  2002       2
          X2: X-Men United  2003       5
              Taking Lives  2004       2
           The Deer Hunter  1978       4


UInt64Index([    8,    28,    30,    58,   108,   175,   187,   191,   197,
               199,
             ...
             17499, 17526, 17537, 17560, 17621, 17622, 17627, 17692, 17703,
             17764],
            dtype='uint64', name='Movie_Id', length=836)

In [92]:
len (get_prev_liked_movies(1987434, verbose= False))


836

In [61]:


reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(
    df[['Cust_Id', 'Movie_Id', 'Rating']][:10000000], reader)

# data.split(n_folds=3)
# trainset = data.build_full_trainset()

# param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE', 'FCP'], cv=3)
# grid_search.fit(data)
# print(grid_search.best_score['RMSE'])
# print(grid_search.best_params['RMSE'])
# print(grid_search.best_score['FCP'])
# print(grid_search.best_params['FCP'])
# print(grid_search.best_score)


svd = SVD()
cross_validate(svd, data, measures=['RMSE'] , cv=5, verbose=True)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8655  0.8665  0.8658  0.8672  0.8666  0.8663  0.0006  
Fit time          69.25   69.40   69.03   65.18   65.34   67.64   1.95    
Test time         23.46   23.96   21.79   22.29   22.25   22.75   0.82    


{'test_rmse': array([0.86554146, 0.86646693, 0.86578002, 0.86718664, 0.8665813 ]),
 'fit_time': (69.251051902771,
  69.39624381065369,
  69.03078699111938,
  65.1816177368164,
  65.33965349197388),
 'test_time': (23.455153226852417,
  23.963589668273926,
  21.785398483276367,
  22.28699827194214,
  22.248989820480347)}

In [62]:
print(sys.getsizeof(df) /1_000_000 , "MB")


646.501725 MB


In [45]:

# data = Dataset.load_from_df(
#     df[['Cust_Id', 'Movie_Id', 'Rating']][:1000000], reader)

# # # getting full dataset
# trainset = data.build_full_trainset()

# algo = SVD()

# algo.fit(trainset)


In [65]:
def recommend_movies(user_id, min_ratings_count=0, n_movies=10 , algo=svd):
    user = df_title.copy()
    user = user.reset_index()
    user = user[~user['Movie_Id'].isin(drop_movie_list)]
    # we remove from user table the movies that the user has already rated from get_prev_liked_movies function
    user = user[~user['Movie_Id'].isin(get_prev_liked_movies(user_id))]
    # print(user['Movie_Id'].isin(get_prev_liked_movies(user_id)).to_string(index=False))
    # print(get_prev_liked_movies(user_id))
    # print(user.head())
    user['Estimated rating'] = user['Movie_Id'].apply(
        lambda x: algo.predict(user_id, x).est)
    user.set_index('Movie_Id', inplace=True)
    
    # print(user.head())
    user = user.join(df_movie_summary)[['Name' , 'Estimated rating', 'count']]
    user = user[user['count'] > min_ratings_count]
    user = user.sort_values('Estimated rating', ascending=False)
    user.rename(columns={"Name": "movie name", "count": "number of ratings" } , inplace=True)
    # print(user.head(n_movies).to_string(index=False))
    print(user.head(n_movies).to_string(index=True))



In [67]:
recommend_movies(2342)


Liked movies by user 2342 are : 
                                             movie name  Estimated rating  number of ratings
Movie_Id                                                                                    
2114                                            Firefly          4.459413              10772
2102                             The Simpsons: Season 6          4.437537               8426
1476                           Six Feet Under: Season 4          4.423333              11829
2452      Lord of the Rings: The Fellowship of the Ring          4.357689             149866
270                          Sex and the City: Season 4          4.340023              36431
1495                                    Alias: Season 1          4.336051              16683
2162                                      CSI: Season 1          4.327326              17013
1795                                     MASH: Season 7          4.324654               5025
1692                                 

In [57]:
# for i in range(3 , 400):
#     print(algo.predict(i, 2452).est)


In [55]:
# df_filterd_1 = df_filterd[:10000000]
df_filterd_1 = df_filterd
# n = 1000000
n = 10000000
# Split train- & testset
df_train = df_filterd_1[:-n]
df_test = df_filterd_1[-n:]

In [57]:
del  df_filterd 
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50546559 entries, 5098 to 83938150
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 819.5 MB


In [26]:
# Create Cust- & movie-id mapping
Cust_id_mapping = {id:i for i, id in enumerate(df_filterd_1['Cust_Id'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(df_filterd_1['Movie_Id'].unique())}


# Create correctly mapped train- & testset
train_Cust_data = df_train['Cust_Id'].map(Cust_id_mapping)
train_movie_data = df_train['Movie_Id'].map(movie_id_mapping)

test_Cust_data = df_test['Cust_Id'].map(Cust_id_mapping)
test_movie_data = df_test['Movie_Id'].map(movie_id_mapping)


# Create sparse matrix from ratings
shape = (len(Cust_id_mapping), len(movie_id_mapping))
train_matrix = coo_matrix((df_train['Rating'].values, (train_Cust_data.astype(int), train_movie_data.astype(int))), shape=shape)
test_matrix = coo_matrix((df_test['Rating'].values, (test_Cust_data.astype(int), test_movie_data.astype(int))), shape=shape)

In [40]:

# Instantiate and train the model
model = LightFM(loss='warp', no_components=20)
model.fit(train_matrix, epochs=5, num_threads=1 , verbose=True)


# Evaluate the trained model
k = 20
print('Train precision at k={}:\t{:.4f}'.format(k, precision_at_k(model, train_matrix, k=k).mean()))
print('Test precision at k={}:\t\t{:.4f}'.format(k, precision_at_k(model, test_matrix, k=k).mean()))

Test precision at k=20:		0.0000


In [39]:
model.predict(np.array([2384]), np.array([234]))
# i still couldnt get the lightfm library to work properly , i will try to fix it later

array([-2.418237], dtype=float32)

In [63]:
# Create user- & movie-id mapping
user_id_mapping = {id:i for i, id in enumerate(df_filterd_1['Cust_Id'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(df_filterd_1['Movie_Id'].unique())}


# Create correctly mapped train- & testset
train_user_data = df_train['Cust_Id'].map(user_id_mapping)
train_movie_data = df_train['Movie_Id'].map(movie_id_mapping)

test_user_data = df_test['Cust_Id'].map(user_id_mapping)
test_movie_data = df_test['Movie_Id'].map(movie_id_mapping)


# Get input variable-sizes
users = len(user_id_mapping)
movies = len(movie_id_mapping)
embedding_size = 10


##### Create model
# Set input layers
user_id_input = Input(shape=[1], name='user')
movie_id_input = Input(shape=[1], name='movie')

# Create embedding layers for users and movies
user_embedding = Embedding(output_dim=embedding_size, 
                           input_dim=users,
                           input_length=1, 
                           name='user_embedding')(user_id_input)
movie_embedding = Embedding(output_dim=embedding_size, 
                            input_dim=movies,
                            input_length=1, 
                            name='item_embedding')(movie_id_input)

# Reshape the embedding layers
user_vector = Reshape([embedding_size])(user_embedding)
movie_vector = Reshape([embedding_size])(movie_embedding)

# Compute dot-product of reshaped embedding layers as prediction
# y = Dot(1, normalize=False)([user_vector, movie_vector])

# # Setup model
# model = Model(inputs=[user_id_input, movie_id_input], outputs=y)
# model.compile(loss='MSE', optimizer='adam')


# # Fit model
# model.fit([train_user_data, train_movie_data],
#           df_train['Rating'],
#           batch_size=2048, 
#           epochs=20,
#           validation_split=0.2,
#           shuffle=True,
#           verbose=True)

# # Test model
# y_pred = model.predict([test_user_data, test_movie_data])
# y_true = df_test['Rating'].values

# #  Compute RMSE
# rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
# print('\n\nTesting Result With Keras Matrix-Factorization: {:.4f} RMSE'.format(rmse))

Epoch 1/20
19745/19745 [==============================] - 143s 7ms/step - loss: 1.6771 - val_loss: 14.5624
Epoch 2/20
19745/19745 [==============================] - 143s 7ms/step - loss: 0.8133 - val_loss: 14.5624
Epoch 3/20
 5086/19745 [======>.......................] - ETA: 1:40 - loss: 0.7640

KeyboardInterrupt: 

In [ ]:
# the tensorflow neural network model is very promising so far , i will try to improve it later
# and first try to create another reccomendation function that maps the data to the embedding layers first and then uses the model to predict the ratings

In [64]:
tf.__version__

'2.5.0'